# David Silver Reinforcement Learning Note

## 目录

- [Lecture 1 Introduction to Reinforcement Learning](#Lecture-1-Introduction-to-Reinforcement-Learning)
- [Lecture 2 Markov Decision Processes](#Lecture-2-Markov-Decision-Processes)
- [Lecture 3 Planning by Dynamic Programming](#Lecture-3-Planning-by-Dynamic-Programming)
- [Lecture 4 Model Free Prediction](#Lecture-4-Model-Free-Prediction)
- [Lecture 5 Model Free Control](#Lecture-5-Model-Free-Control)
- [Lecture 6 Value Function Approximation](#Lecture-6-Value-Function-Approximation)
- [Lecture 7 Policy Gradient](Lecture-7-Policy-Gradient)
- [Lecture 8 Integrating Learning and Planning](Lecture-8-Integrating-Learning-and-Planning)
- [Lecture 9 Exploration and Exploitation](Lecture-9-Exploration-and-Exploitation)
- [Lecture 10 Classic Games](Lecture-10-Classic-Games)

## 大致介绍

介绍下课程推进的逻辑：

第一章介绍基础概念，用着重分清楚`Reward`,`value function`和`q function`的区别。

第二章介绍MDP，MDP是对环境的一种建模，即state将以一种什么样的方式推进是已知的。重点理解`Bellman Expectation Equation`和`Bellman Optimality Equation`，二者本质上都是递推公式，给动态规划打下基础。

第三章介绍如何处理MDP，分为两部：`planning`和`controlling`。`planning`代表如何评估$v_{\pi}$，`controlling`代表如何根据评估，找到最优的策略$\pi_*$。解决方案包括`policy iteration`和`value iteration`。

第四章介绍环境未知的情况下（model free），如何做planning。有两种方法：`MC（Monte-Carlo）`和`TD（Temporal Difference）`。

第五章介绍环境未知的情况下，如何做`controlling`。针对policy的有：`On-Policy Monte-Carlo Control`，`On-Policy Sarsa Control`，`Off-Policy Q-learning`。

## Lecture 1 Introduction to Reinforcement Learning

**奖励（Reward）**：$R_t$是$t$时刻的奖励。agent的任务是最大化**累积的（cumulative）**奖励。

**观察（Observation）**：$O_t$，$t$时刻对环境的观察。

**行动（Action）**：$A_t$，$t$时刻采取的行动。

**历史（History）**：是一个序列的观察/奖励/行动（某时刻之前的所有时刻）。$H_t = A_1, O_1, R_1, ... A_t, O_t, R_t$。

**状态（State）**：通过对历史信息的总结，决定接下来要发生什么。$S_t = f(H_t)$

**环境状态（Environment State）**：某个环境的状态，$S_t^e$。

**代理环境（Agent State）**：agent内部的环境，$S_t^a$。

**马尔科夫状态（Markov state）**：状态$S_t$是马尔科夫状态，如果满足：$P[S_{t+1}|S_t] = P[S_{t+1}| S_1, ..., S_t]$，到这一步可以把**历史**丢掉了，只要每一步的状态即可。

**全观测环境（Full observability）**：$O_t = S_t^a = S_t^e$。

**部分观测环境（Partial observability）**：$S_t^a ≠ S_t^e$。

**Policy**：Policy是agent的行为，是state到action的映射。确定形式：$a=\pi (s)$；随机形式：$\pi(a|s)=P[A_t=a|S_t=s]$。

**Value**：Value是对未来reward的预测：$v_{\pi}(s) = E_{\pi}[R_{t+1}+\gamma R_{t+2}+\gamma ^2R_{t+3}+...|S_t=s]$

**Model**：Model是对未来环境的预测：$\textbf{P}$预测下一个state：$\textbf{P}^a_{ss'}=P[S_{t+1}=s'|S_t=s,A_t=a]$，$\textbf{R}$预测下一个即时的奖励：$\textbf{R}_s^a=E[R_{t+1}|S_t=s,A_t=a]$。

**Agent 分类**：
- model based（model + policy and/or + value function）
- model free:
    - value based (value function)
    - policy based (policy)
    - actor critic (policy + value function)
    
![](https://github.com/applenob/rl_learn/raw/master/res/agent.png)

## Lecture 2 Markov Decision Processes

**Markov Decision Process（MDP）**：思想：`The future is independent of the past given the present`，即某个时刻，agent所处在的状态，只和它上一个时刻所处的状态相关。是对**环境（实际上是state）**的一种描述。$P[S_{t+1}|S_t]=P[S_{t+1}|S_1,...,S_t]$

**状态转移矩阵（State Transition Matrix）**：$\textbf{P}$其中，$\textbf{P}_{ss'}=P[S_{t+1}=s'|S_t=s]$。

**Markov Process**：由简单到复杂：
- 包含$<\textbf{S}, \textbf{P}>$，即**状态集合**和**状态转移矩阵**。
- MRP：包含$<\textbf{S}, \textbf{P}, \textbf{R}, \gamma>$，即增加了**奖励函数**和**折扣系数**，$ \textbf{R}_s = E[\textbf{R}_{t+1} | S_t = s]$。
- 引入$G_t=R_{t+1} + \gamma R_{t+2} + ... =\sum^{\infty}_{k=0}\gamma^kR_{t+k+1}$，注意Value是期望，G是针对一个sample，$v(s) = E [G_t | S_t = s]$。
- MDP:包含$<\textbf{S}, \textbf{A}, \textbf{P}, \textbf{R}, \gamma>$，增加了$\textbf{A}$，是有限的动作集合。$\textbf{P}_{ss'}^a=P[S_{t+1}=s'|S_t=s,A_t=a]$，$\textbf{R}_s^a=E[R_{t+1}|S_t=s,A_t=a]$。

**$G_t$ vs. $v(s)$**：再次注意二者的区分：$G_t$刻画的是针对一个sample序列（episode），在某一时刻$t$可以获得的长期奖励，这根据你的实验是可以计算出来的，因为你的试验中，每走一步会有一个$R_t$，$G_t=R_{t+1} + \gamma R_{t+2} + ... =\sum^{\infty}_{k=0}\gamma^kR_{t+k+1}$当然是可以计算的。$v_s$描述的是对某一个状态$s$的评价，我想不想经过这个状态。对状态的评价可以通过我在所有的episode中，只要状态是$s$的，我都计算一个$G_t$，然后把这些$G_t$求平均，就可以拿来评估这个状态的好坏，理论上，求平均应该是求期望。

**MRP的Bellman Equation**：
- 针对奖赏，有$$v(s) = E[G_t|S_t = s]\\=E[R_{t+1}+\gamma v(S_{t+1})|S_t = s]\\=\textbf{R}_s+\gamma\sum_{s'\in S}\textbf{P}_{ss'}v(s')$$
- 矩阵形式：$v = R + \gamma Pv$。
- 直接求解，时间复杂度是$O(n^3)$。

**Policy**：$\pi$是给定状态之后动作的分布，即$\pi(a|s)=P[A_t=a|S_t=s]$，policy可以完整地描述agent的行为，policy是独立于时间的。

**MDP的state-value函数**：$v_{\pi}(s)=E[G_t|S_t=s]$，即在状态$s$下，遵从policy-$\pi$可以带来的长期奖励的期望。

**MDP的action-value函数**：$q_{\pi}(s,a) = E_{\pi}[G_t|S_t=s, A_t=a]$，即在状态$s$下，采取了动作$a$，然后遵从policy-$\pi$可以带来的长期奖励的期望。这是我们真正在乎的，我们需要知道哪个action更好！

**q vs. v**：简单地说，$q$用来衡量某一个state下，采取某一个特定的action有多好，$v$用来衡量某一个state有多好。

**MDP的Bellman Expectation Equation**：
- $v_{\pi}(s)=E_{\pi}[R_{t+1}+\gamma v_{\pi}(S_{t+1})|S_t=s]$
- $q_{\pi}(s,a)=E_{\pi}[R_{t+1}+\gamma q_{\pi}(S_{t+1}, A_{t+1})|S_t=s]$
- $v_{\pi}(s)=\sum_{a\in A}\pi (a|s)q_{\pi}(s,a)$
- $q_{\pi}(s,a)=\textbf{R}_s^a+\gamma\sum_{s'\in S}\textbf{P}_{ss'}^av_{\pi}(s')$
- 最终：$v_{\pi}(s)=\sum_{a\in A}\pi (a|s)(\textbf{R}_s^a+\gamma\sum_{s'\in S}\textbf{P}_{ss'}^av_{\pi}(s'))$
- 最终：$q_{\pi}(s,a)=\textbf{R}_s^a+\gamma\sum_{s'\in S}\textbf{P}_{ss'}^a\sum_{a'\in A}\pi (a'|s')q_{\pi}(s',a')$

**MDP的Bellman Expectation Equation的矩阵形式**：
- $v_{\pi} = \textbf{R}^{\pi}+\gamma\textbf{P}^{\pi}v_{\pi}$

**最优化的函数**：
- $v_*(s)=\underset{\pi}{max}\;v_{\pi}(s)$
- $q_*(s,a) = \underset{\pi}{max}\;q_{\pi}(s, a)$

**policy的比较**：如果一个policy在所有state下，v都比比另一个policy大，那么可以说这个policy比另一个policy好。即，$\pi > \pi',\;if\;v_{\pi}(s)≥v_{\pi'}(s),\forall s$

**对于所有的MDP，都有**：
- 存在最优policy，使得$\pi_*≥\pi,\;\forall \pi$；
- $v_{\pi_*} = v_*(s)$；
- $q_{\pi_*}(s,a) = q_*(s,a)$。

**假设已经找到$q_*(s,a)$，可以马上获得$\pi_(a|s)$**：$\pi_*(a|s)=\left\{\begin{matrix}
1\;\;if\; a=\underset{a\in A}{argmax}\;q_*(s,a)\\ 
0\;\;\;\;\;\;\;\;\;\;\;otherwise
\end{matrix}\right.$

**Bellman Optimality Equation **：
- for $v_*$：$v_*(s)=\underset{a}{max}\;q_*(s,a)$
- for $q_*$：$q_*(s,a)=\textbf{R}_s^a+\gamma\sum_{s'\in S}\textbf{P}_{ss'}^av_*(s')$

**解决Bellman Optimality Equation的方法**：
- Bellman Optimality Equation是非线性的，不能直接计算；
- 迭代求解的方法：
    - MDP：
        - Value Iteration
        - Policy Iteration
    - Model Free：
        - Q-learning
        - Sarsa

**Bellman Equation总结**：
- Bellman Equation可以理解成某一时刻在某一状态下的value可以拆分成即时奖励和一下一个时刻开始所有可能的状态的价值的期望。
- 它本身是一个递推公式，可以设计一个`backup`表格，不断进行更新。

## Lecture 3 Planning by Dynamic Programming


**动态规划（Dynamic Programming）**：
- 给定MDP，用于MDP的**planning**；
- **prediction**：
    - 输入：MDP：$<\textbf{S}, \textbf{A}, \textbf{P}, \textbf{R}, \gamma>$和policy：$\pi$；或者MRP：$<\textbf{S}, \textbf{P}^{\pi}, \textbf{R}^{\pi}, \gamma>$。
    - 输出：$v_{\pi}$的值。
- **control**:
    - 输入：MDP：$<\textbf{S}, \textbf{A}, \textbf{P}, \textbf{R}, \gamma>$。
    - 输出：最优化的policy：$\pi_*$

**Iterative Policy Evaluation**：
- 评估一个给定的policy：$\pi$。
- 使用Bellman Expectation Equation解决。
- 迭代过程：$v_1\to v_2\to ...\to v_{\pi}$
- 使用synchronous backups，在第$k+1$步迭代：
    - 对所有的状态$s\in S$
    - 通过$v_k(s')$更新$v_{k+1}(s)$，矩阵形式：$\textbf{v}^{k+1}=\textbf{R}^{\pi}+\gamma\textbf{P}^{\pi}\textbf{v}^k$，其中$s'$是$s$的后继状态。

**Policy Iteration**：
- 目标：寻找最优policy：$\pi_*$。
- 给定一个policy：$\pi$：
    - Evaluate这个policy：$v_{\pi}(s)=E[G_t|S_t=s]=E[R_{t+1}+\gamma R_{t+2}+...|S_t = s]$
    - Improve这个policy：$\pi'=greedy(v_{\pi})$
- 最终会收敛到$\pi_*$。

![](https://github.com/applenob/rl_learn/raw/master/res/policy_evaluation.png)

**Value Iteration**：
- 目标：寻找最优policy：$\pi_*$。
- 使用Bellman Optimality Equation解决。
- 迭代过程：$v_1\to v_2\to ...\to v_*$。
- 使用synchronous backups，在第$k+1$步迭代：
    - 对所有的状态$s\in S$
    - 通过$v_k(s')$更新$v_{k+1}(s)$，矩阵形式：$\textbf{v}_{k+1}=\underset{a \in A}{max}\;\textbf{R}^a+\gamma\textbf{P}^a\textbf{v}_k$。
- 中间过程没有特定的policy。

**Policy Iteration vs. Value Iteration**：
- Policy Iteration: $v_1\to \pi \to v_2$
- Value Iteration：$v_1\to v_2$
- 中间的$v_i$可能没有对应的policy。


## Lecture 4 Model Free Prediction

**Monte-Carlo 强化学习**：
- model free：没有mdp或者reward。
- 适用于所有的episode都会结束的情形。

**Monte-Carlo Policy Evaluation**：
- Goal：使用policy：$\pi$，从经验序列（episode）：$S_1,A_1,R_2,...,S_k$中学习价值$v_{\pi}$。
- 回顾$v_{\pi}=E_{\pi}(s) = E_{\pi}[G_t|S_t=s]$，Monte-Carlo**使用平均返回值代替期望返回值**。
- 有以下两种方法。

**First-Vist Monte-Carlo Policy Evaluation**：
- 评估某个状态$s$。
- 在某一个episode中，第一次发生状态$s$的是时间$t$。
- 计数器自增：$N(s) \leftarrow  N(s)+1$
- $S(s)\leftarrow S(s) + G_t$
- 计算价值函数：$V(s) \leftarrow  S(s)/N(s)$

**Every-Visit Monte-Carlo Policy Evaluation**：类似于上面，只不过是每次出现状态$s$，都要纳入计算。

**增量计算均值**：$\mu=\frac{1}{k}\sum_{j=1}^kx_j\\=\frac{1}{k}(x_k+(k-1)\mu_{k-1})\\=\mu_{k-1}+\frac{1}{k}(x_k-\mu_{k-1})$

**增量式的Monte-Carlo更新**：
- 对于每个Episode：$S_1, A_1, R_2, ..., S_T$结束完以后，增量地更新$V(s)$。
- $N(S_t) \leftarrow N(S_t) + 1$
- $V(S_t)\leftarrow V(S_t)+\frac{1}{N(S_t)}(G_t-V(S_t))$
- 可以写成$V(S_t)\leftarrow V(S_t)+\alpha (G_t-V(S_t))$

**瞬时分差Temporal-Difference Learning**：
- model free：没有mdp或者reward。
- 适用于不完整的episode的情形，使用bootstrapping（bootstrapping代表更新的时候需要估计期望，而不是使用真的反馈）。
- TD通过猜测来更新猜测。

**TD**：
- Goal：使用policy：$\pi$，中学习价值$v_{\pi}$。
- 相比MC只是更新的式子不同：$V(S_t)\leftarrow V(S_t)+\alpha (R_{t+1}+\gamma V(S_{t+1})-V(S_t))$
- 其中，$R_{t+1} + γV(S_{t+1})$ 被称为`TD target`。
- $δ_t = R_{t+1} + γV(S_{t+1}) − V(S_t)$ 称为`TD error`。

**MC vs. TD**：
- 用通俗的语言描述就是，对于某个state的value更新，MC会把这个state后面所有的state都计算R，用来计算G；而TD只会计算当前下一个状态的R，其他后面的R不算，采用当前估计的value来替代。
- MC有高方差，0偏差，对初始值不敏感，使用sample，不使用bootstrapping。
- TD往往更高效，且对初始值敏感，使用sample，使用bootstrapping。

**n-step TD**：
![](https://github.com/applenob/rl_learn/raw/master/res/n_step_td.png)

**TD($\lambda$)**：
- Forward-view：
![](https://github.com/applenob/rl_learn/raw/master/res/td_lambda.png)
- 每个n-step的$G_t$加权求和，权重：$(1-\lambda)\lambda^{n-1}$，$G_t^{\lambda}=(1-\lambda)\sum^{\infty}_{n=1}{\lambda}^{n-1}G_t^{n}$
- $V(S_t)\leftarrow V(S_t) + \alpha(G_t^{\lambda}-V(S_t))$

**Forward-view和Backward-view**：
- Forward-view只能计算完整的episodes。
- Backward-view可以计算不完整的episodes。
- Forward-view提供了理论。
- Backward-view提供了机制。
- Forward-view倾向于最高频的状态。

**Eligibility Traces**：
- Eligibility Traces结合了高频的倾向和最近的倾向。
- $E_0(s) = 0$
- $E_t(s) = \gamma\lambda E_{t-1}(s) + \textbf{1}(S_t=s)$，累加代表了高频倾向；但之前的时刻会衰减代表了最近的倾向。

**Backward-view TD($\lambda$)**：
- $\delta_t=R_{t+1}+\gamma V(S_{t+1} - V(S_t))$
- $V(s)\leftarrow V(s)+\alpha \delta_t E_t(s)$

## Lecture 5 Model Free Control

**On-policy learning vs. Off-policy learning**：
- on代表在任务中学习，从采样的$\pi$中学习$\pi$。（自学）
- off代表在某种基础上学习，从采样的$\mu$中学习$\pi$。（有人示范）

**On-Policy Monte-Carlo Control**：
- policy evaluation：Monte-Carlo policy evaluation，$Q≈q_{\pi}$。
- policy improvement：$\epsilon$-greedy policy improvement
![](https://github.com/applenob/rl_learn/raw/master/res/mc_policy_iteration.png)

**为什么使用Q？**：V的`greedy policy improvement`依赖于MDP，而Q的则不需要。

**$\epsilon$-greedy policy improvement**：
- $\pi(a|s)=\left\{\textbf{}\begin{matrix}\epsilon/m+1-\epsilon\;\;\;if\;a*=\underset{a\in A}{argmax}Q(s,a)\\ \epsilon/m\;\;\;otherwise\end{matrix}\right.$
- 即，有$\epsilon$的可能是随机选择。

**Greedy in the Limit with Infinite Exploration (GLIE)**：
- 希望上面提到的$\epsilon$是衰减的，即到最后的时候，直接采用贪婪选择。

**On-Policy Sarsa Control**：
![](https://github.com/applenob/rl_learn/raw/master/res/sarsa_2.png)
![](https://github.com/applenob/rl_learn/raw/master/res/sarsa.png)
- 所谓sarsa即，使用$\epsilon$-greedy，根据$s$，选择$a$，获得奖励$r$，和新的状态$s'$，再使用$\epsilon$-greedy，选择$a'$

**n-step Sarsa**：
![](https://github.com/applenob/rl_learn/raw/master/res/n_step_sarsa.png)

**Forward View Sarsa($\lambda$)**：
- $q_t^{\lambda}=(1-\lambda)\sum_{n=1}^{\infty}\lambda ^{n-1}q_t^{(n)}$
- $Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha(q_t^{\lambda}-Q(S_t, A_t))$

**Backward View Sarsa($\lambda$)**：
- $E_0(s, a)= 0$
- $E_t(s, a)=\gamma\lambda E_{t-1}(s, a)+\mathbf{1}(S_t=s,A_t=a)$
- $\delta_t = R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)$
- $Q(s, a)\leftarrow Q(s, a)+\alpha\delta_tE_t(s,a)$
![](https://github.com/applenob/rl_learn/raw/master/res/sarsa_lamda.png)

**Off-Policy Learning**：behaviour policy：$\mu(a|s)$

**Q-Learning**：
- off-policy。
- 不需要importance采样。
- 下一个动作从behaviour policy中选择：$A_{t+1}\sim \mu(\cdot|S_t)$
- 备选的动作：$A'\sim \pi(\cdot|S_t)$，用于下面式子的计算。
- 更新$Q(S_t, A_t)\leftarrow Q(S_t, A_t)+\alpha(R_{t+1}+\gamma Q(S_{t+1}, A')-Q(S_t, A_t))$
![](https://github.com/applenob/rl_learn/raw/master/res/q-learning.png)

**MDP vs. Model Free**：
![](https://github.com/applenob/rl_learn/raw/master/res/dp_td.png)
![](https://github.com/applenob/rl_learn/raw/master/res/dp_td2.png)


## Lecture 6 Value Function Approximation

**对于大型的MDP**：
- 之前我们通过一张大表，表格上有$V(s)$或者$Q(s,a)$。
- 对于大型的MDP问题，表格不再适用，于是可以估计价值函数。
- $\hat v(s, w) \approx v_{\pi}(s)$
- $\hat q(s,a,w) \approx q_{\pi}(s, a)$
- 使用MC或者TD来更新$w$。

**Value Function Approximation的种类**：
![](https://github.com/applenob/rl_learn/raw/master/res/fun_app.png)

**随机梯度下降**：
- 先假设有个已知的$v_{\pi}(S)$
- 目标：$J(w) = E_{\pi}[(v_{\pi}(S)-\hat v(S,w))^2]$
- 梯度下降：$\triangle w = -\frac{1}{2}\alpha\triangledown_wJ(w)=\alpha E_{\pi}[(v_{\pi}(S)-\hat v(S,w))\triangledown_w \hat v(S,w)]$
- 随机梯度下降只用sample其中一个：$\triangle w =\alpha (v_{\pi}(S)-\hat v(S,w))\triangledown_w \hat v(S,w)$

**特征向量（Feature Vector）**：
- 用来表示一个state。
- $x(S) = \begin{pmatrix}x_1(S)\\ ...\\ x_n(S)\end{pmatrix}$

**线性特征组合**：
- 用特征的线性组合去表示value function。
- $\hat v(S, w) = x(S)^Tw=\sum^n_{j=1}x_j(S)w_j$
- $\triangle w = \alpha(v_{\pi}(S)-\hat v(S, w))x(S)$
- 也就是 update = 步长×预测误差×特征值

**增量预测算法（Incremental Prediction Algorithms）**：
- 回到“$v_{\pi}(S)$是未知的”这个事实。
- 我们使用以下的方法，不同的方法使用不同的量去替代$v_{\pi}(S)$。
- $MC$：使用$G_t$，$\triangle w = \alpha(G_t - \hat v(S_t, w))\triangledown_w\hat v(S_t, w)$
- $TD(0)$：使用$R_{t+1}+\gamma \hat v(S_{t+1}, w)$，$\triangle w = \alpha(R_{t+1}+\gamma \hat v(S_{t+1}, w) - \hat v(S_t, w))\triangledown_w\hat v(S_t, w)$
- $TD(\lambda)$：使用$G_t^{\lambda}$，$\triangle w = \alpha(G_tt^{\lambda} - \hat v(S_t, w))\triangledown_w\hat v(S_t, w)$

**Control with Value Function Approximation**：
![](https://github.com/applenob/rl_learn/raw/master/res/control_VFA.png)

**Action-Value Function Approximation**：和Value Function Approximation类似。

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

## Lecture 7 Policy Gradient

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

## Lecture 8 Integrating Learning and Planning

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

## Lecture 9 Exploration and Exploitation

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

## Lecture 10 Classic Games

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：

****：